<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda-dev/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%matplotlib inline
!pip3 install openpifpaf

import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True

from openpifpaf_animalplugin import AnimalPoseEstimation
#from openpifpaf_sdaplugin import SDA
import coco_adapt as coco_adapt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Dataset download and manipulation

In [18]:
!rm -rf plugins
!rm -rf openpifpaf_animalplugin
!git clone --branch sda-dev https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git plugins
!mv plugins/openpifpaf_animalplugin/ openpifpaf_animalplugin
#!rm -rf data-animalpose/
#!mkdir data-animalpose
#!gdown "https://drive.google.com/drive/folders/1xxm6ZjfsDSmv6C9JvbgiGrmHktrUjV5x" -O data-animalpose --folder 
#!unzip data-animalpose/images.zip -d data-animalpose/
#!rm data-animalpose/images.zip

Cloning into 'plugins'...
remote: Enumerating objects: 1858, done.
remote: Counting objects: 100% (1713/1713), done.
remote: Compressing objects: 100% (1490/1490), done.
remote: Total 1858 (delta 182), reused 1694 (delta 166), pack-reused 145
Receiving objects: 100% (1858/1858), 70.19 MiB | 24.76 MiB/s, done.
Resolving deltas: 100% (227/227), done.


In [21]:
coco_adapt.adapt_to_coco()
config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES

{'animal': openpifpaf.plugins.animalpose.animal_kp.AnimalKp,
 'apollo': openpifpaf.plugins.apollocar3d.apollo_kp.ApolloKp,
 'cifar10': openpifpaf.plugins.cifar10.datamodule.Cifar10,
 'cocodet': openpifpaf.plugins.coco.cocodet.CocoDet,
 'cocokp': openpifpaf.plugins.coco.cocokp.CocoKp,
 'crowdpose': openpifpaf.plugins.crowdpose.module.CrowdPose,
 'nuscenes': openpifpaf.plugins.nuscenes.nuscenes.NuScenes,
 'posetrack2018': openpifpaf.plugins.posetrack.posetrack2018.Posetrack2018,
 'posetrack2017': openpifpaf.plugins.posetrack.posetrack2017.Posetrack2017,
 'cocokpst': openpifpaf.plugins.posetrack.cocokpst.CocoKpSt,
 'wholebody': openpifpaf.plugins.wholebody.wholebody.Wholebody,
 'custom_animal': openpifpaf_animalplugin.dataset.AnimalPoseEstimation}

# Animal Pose Estimation Dataset and Plugin

In [26]:
dataset = AnimalPoseEstimation()

# Semantic Data Augmentation

In [ ]:
sda = SDA(output_dir='data-animalpose/sda')

sda(dataset[0][0], dataset[0][1])

TypeError: 'AnimalPoseEstimation' object is not subscriptable

# Training

In [27]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k30 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=50 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 2 \
    --batch-size 11

INFO:__main__:neural network device: cpu (CUDA available: False, count: 0)
INFO:openpifpaf.network.basenetworks:shufflenetv2k30: stride = 16, output features = 2048
INFO:openpifpaf.network.losses.multi_head:multihead loss: ['custom-animal.cif.c', 'custom-animal.cif.vec', 'custom-animal.cif.scales', 'custom-animal.caf.c', 'custom-animal.caf.vec', 'custom-animal.caf.scales'], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
INFO:openpifpaf.logger:{'type': 'process', 'argv': ['/usr/local/lib/python3.9/dist-packages/openpifpaf/train.py', '--dataset', 'custom_animal', '--basenet=shufflenetv2k30', '--lr=0.00002', '--momentum=0.95', '--epochs=50', '--lr-decay', '160', '260', '--lr-decay-epochs=10', '--weight-decay=1e-5', '--weight-decay=1e-5', '--val-interval', '10', '--loader-workers', '2', '--batch-size', '11'], 'args': {'output': 'outputs/shufflenetv2k30-230425-073221-custom_animal.pkl', 'disable_cuda': False, 'ddp': False, 'local_rank': None, 'sync_batchnorm': True, 'quiet': False, 'debug': False, 'log_sta

In [ ]:
import zipfile
from google.colab import drive, files
import os

# create a zip file of the folder you want to save
folder_path = '/outputs'
zip_path = '/content/myfolder.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file))

# save the zip file to your Google Drive
drive_path = '/content/drive/My Drive/myfolder.zip'
!cp '{zip_path}' '{drive_path}'